In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fw_v1.interface import vlstm_fw_torch
from src.vlstm_fw_v1.interface import vlstm_fw_cuda

# Commit: 59a91237f7e6733247aa5d2b3da8a66970e7bf54
# ptxas info    : 343 bytes gmem
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwIfLi4ELi8ELi8EEEvPT_S3_S3_S3_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwIfLi4ELi8ELi8EEEvPT_S3_S3_S3_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 66 registers, 400 bytes cmem[0]
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwI6__halfLi4ELi8ELi8EEEvPT_S4_S4_S4_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwI6__halfLi4ELi8ELi8EEEvPT_S4_S4_S4_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 72 registers, 400 bytes cmem[0]
# ptxas info    : Compiling entry function '_ZN5vlstm7kernels8vlstm_fwI13__nv_bfloat16Li4ELi8ELi8EEEvPT_S4_S4_S4_iiii' for 'sm_89'
# ptxas info    : Function properties for _ZN5vlstm7kernels8vlstm_fwI13__nv_bfloat16Li4ELi8ELi8EEEvPT_S4_S4_S4_iiii
#     24 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
# ptxas info    : Used 63 registers, 400 bytes cmem[0]

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fw_v1/build.ninja...
Building extension module vlstm_fw_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_fw_v1...


## CUDA vLSTM forward

In [3]:
S = 16 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1)# / 100.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[ 0.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,  0.6000,  0.7000],
           [ 0.8000,  0.9000,  1.0000,  1.1000,  1.2000,  1.3000,  1.4000,  1.5000],
           [ 1.6000,  1.7000,  1.8000,  1.9000,  2.0000,  2.1000,  2.2000,  2.3000],
           [ 2.4000,  2.5000,  2.6000,  2.7000,  2.8000,  2.9000,  3.0000,  3.1000],
           [ 3.2000,  3.3000,  3.4000,  3.5000,  3.6000,  3.7000,  3.8000,  3.9000],
           [ 4.0000,  4.1000,  4.2000,  4.3000,  4.4000,  4.5000,  4.6000,  4.7000],
           [ 4.8000,  4.9000,  5.0000,  5.1000,  5.2000,  5.3000,  5.4000,  5.5000],
           [ 5.6000,  5.7000,  5.8000,  5.9000,  6.0000,  6.1000,  6.2000,  6.3000],
           [ 6.4000,  6.5000,  6.6000,  6.7000,  6.8000,  6.9000,  7.0000,  7.1000],
           [ 7.2000,  7.3000,  7.4000,  7.5000,  7.6000,  7.7000,  7.8000,  7.9000],
           [ 8.0000,  8.1000,  8.2000,  8.3000,  8.4000,  8.5000,  8.6000,  8.7000],
           [ 8.8000,  8.9000,  9.0000,  9.1000,  9.2000,  9.3000,

In [5]:
# pytorch version
rs, log_fgm, igm, matD = vlstm_fw_torch(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igs,
    fgate_preact=fgs,
)
rs, rs.shape

(tensor([[[[0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030, 0.0030],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],
           [0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100],

In [6]:
# pytorch version QKV product (To test if this is still correct after changes to the code.)
# at some point we have to compare to the vlstm_fw_torch version.
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[0.0045, 0.0045, 0.0045, 0.0045, 0.0045, 0.0045, 0.0045, 0.0045],
           [0.0147, 0.0147, 0.0147, 0.0147, 0.0147, 0.0147, 0.0147, 0.0147],
           [0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250, 0.0250],
           [0.0352, 0.0352, 0.0352, 0.0352, 0.0352, 0.0352, 0.0352, 0.0352],
           [0.0454, 0.0454, 0.0454, 0.0454, 0.0454, 0.0454, 0.0454, 0.0454],
           [0.0557, 0.0557, 0.0557, 0.0557, 0.0557, 0.0557, 0.0557, 0.0557],
           [0.0659, 0.0659, 0.0659, 0.0659, 0.0659, 0.0659, 0.0659, 0.0659],
           [0.0762, 0.0762, 0.0762, 0.0762, 0.0762, 0.0762, 0.0762, 0.0762],
           [0.0864, 0.0864, 0.0864, 0.0864, 0.0864, 0.0864, 0.0864, 0.0864],
           [0.0966, 0.0966, 0.0966, 0.0966, 0.0966, 0.0966, 0.0966, 0.0966],
           [0.1069, 0.1069, 0.1069, 0.1069, 0.1069, 0.1069, 0.1069, 0.1069],
           [0.1171, 0.1171, 0.1171, 0.1171, 0.1171, 0.1171, 0.1171, 0.1171],
           [0.1274, 0.1274, 0.1274, 0.1274, 0.1274, 0.1274, 0.1274, 0.1274],

In [7]:
rs_torch = (qs @ ks.transpose(-1, -2) * torch.tril(torch.ones((B, NH, S, S))).to(device=DEVICE, dtype=DTYPE)) @ vs
rs_torch, rs_torch.shape

(tensor([[[[0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018],
           [0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047],
           [0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088],
           [0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142],
           [0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209],
           [0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288],
           [0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381],
           [0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486],
           [0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604],
           [0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735],
           [0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878],
           [0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035],

In [8]:
igs.shape, igs.squeeze(-1).shape

(torch.Size([1, 1, 16, 1]), torch.Size([1, 1, 16]))

In [9]:
# cuda kernel
rs_cuda, matC = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))
rs_cuda, rs_cuda.shape

before kernel dispatch - float32!
B: 1, NH: 1, S: 16, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5700
In Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


(tensor([[[[0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003],
           [0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018],
           [0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047, 0.0047],
           [0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088, 0.0088],
           [0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142, 0.0142],
           [0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209, 0.0209],
           [0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288, 0.0288],
           [0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381, 0.0381],
           [0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486, 0.0486],
           [0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604, 0.0604],
           [0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735, 0.0735],
           [0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878, 0.0878],
           [0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035, 0.1035],

In [10]:
rs_torch - rs_cuda

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.1176e-08,  1.1176e-08,  1.1176e-08,  1.1176e-08,  1.1176e-

In [11]:
matC.shape

torch.Size([1, 1, 16, 16])

In [12]:
matC

tensor([[[[ 1.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6867,  2.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.3735,  1.6867,  3.0000,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0602,  1.3735,  2.6867,  4.0000,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.2530,  1.0602,  2.3735,  3.6867,  5.0000,    -inf,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.5663,  0.7470,  2.0602,  3.3735,  4.6867,  6.0000,    -inf,    -inf,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [-0.8796,  0.4337,  1.7470,  3.0602,  4.3735,  5.6867,

In [13]:
matD

tensor([[[[ 1.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.6867,  2.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.3735,  1.6867,  3.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [ 0.0602,  1.3735,  2.6867,  4.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-0.2530,  1.0602,  2.3735,  3.6867,  5.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-0.5663,  0.7470,  2.0602,  3.3735,  4.6867,  6.0000,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-0.8796,  0.4337,  1.7470,  3.0602,  4.3735,  5.6867,

In [14]:
matC - matD

tensor([[[[ 0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf,         inf,         inf,         inf,         inf,
                   inf,         inf],
          [ 5.9605e-08,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf,         inf,         inf,         inf,         inf,
                   inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         inf,         inf,         inf,         inf,         inf,         inf,
                   inf,         inf],
          [ 5.9605e-08,  1.1921e-07,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         inf,         inf,         inf,         inf,         inf,         inf,
                   inf,         inf],
          [ 1.1921e-07,  2.3842e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00, 